In [2]:
import numpy as np
from numpy import linalg
from scipy import linalg as splinalg
import matplotlib.pyplot as plt
from scipy import sparse as sp
import scipy.sparse.linalg
from functools import reduce
import itertools
from scipy import linalg
from scipy.linalg import expm, logm
from scipy.special import comb
from itertools import combinations_with_replacement, product
from collections import Counter
import copy
from scipy.linalg import ishermitian
import time
from scipy.integrate import solve_ivp

In [51]:
params = {}

params['N_particles'], params['N_sites'] = 2, 10
params['N_region1'], params['N_region2'] = 1, 2

params['n_max'] = 2

params['J'], params['U'] = 1, 0

In [52]:
params['dim'] = 0 

def dimension(params, **kwargs):
    '''
    For k identical bosonic particles on N lattice site, 
    Changes the dim.
    '''
    N, k, dim = params['N_sites'], params['N_particles'], int(params['dim']) 
    params['dim'] = int(comb(N+k-1, k)) 
    pass

dimension(params)

In [53]:
def normalizeWF(psi,**kwargs):
    shape, dtype = psi.shape, psi.dtype
    NWF = psi
    if np.array_equal(psi, np.zeros(shape, dtype = dtype)) == True:
        NWF = psi
    elif np.vdot(psi, psi) == 0:
        NWF = psi
    else:
        NWF = psi/(np.sqrt(np.vdot(psi, psi)))
    return NWF

In [54]:
def creationOp(params, **kwargs):
    '''
    Returns bosonic creation operator.
    To construct annihilation operator, take transpose of this matrix.
    '''
    A = sp.diags(np.sqrt(np.arange(1, params['n_max']+1, 1)), -1, format='csr')
    return A

def hopOp(params, site_no, **kwargs):
    '''
    Hop_op term
    '''
    createOp = creationOp(params)
    I = sp.eye(params['n_max']+1, format='csr')
    lst = [I for _ in range(params['N_sites'])]
    lst[site_no], lst[site_no + 1] = createOp, createOp.conj().T
    return reduce(sp.kron, lst)

def numOp(params, site_no, **kwargs):
    createOp = creationOp(params)
    I = sp.eye(params['n_max']+1, format='csr')
    lst = [I for _ in range(params['N_sites'])]
    lst[site_no] = createOp@createOp.conj().T
    nop = reduce(sp.kron, lst)
    lst[site_no] = (createOp@createOp.conj().T)@(createOp@createOp.conj().T)
    return nop, reduce(sp.kron, lst)

In [55]:
def Prod_OccupBasis(params, **kwargs):
    '''
    Generates all combinations using product from itertools.
    Returns: valid_combinations under the k-constraint (particle
    number conservation) and all combinations.
    '''
    n, N, k = params['n_max']+1, params['N_sites'], params['N_particles']
    all_combinations = dict(enumerate(itertools.product(range(n), repeat=N)))
    valid_combinations = dict(filter(lambda x: sum(x[1]) == k, all_combinations.items()))
    return valid_combinations, all_combinations
# params['truncationParam_n'], params['N'], params['k'] = 2, 2, 2

def projectionMatrix(params, **kwargs):
    '''
    Creates a projection matrix whose elements are non-zero
    for the indices of the occup_states obeying k-constraint.
    '''
    valid_combinations, all_combinations = Prod_OccupBasis(params)
    rows, cols = len(valid_combinations), len(all_combinations)
    PM = sp.csc_matrix((rows, cols))
    for i, key in enumerate(list(reversed(valid_combinations.keys()))):
        PM[i, key] = 1.0
    return PM

In [77]:
def two_siteBJJHamil(params, **kwargs):
    '''
    Returns two-site bosonic Josephson junction hamiltonian
    '''
    H = sp.csc_matrix(((params['n_max']+1)**(params['N_sites']), (params['n_max']+1)**(params['N_sites'])))
    I = sp.eye(params['n_max']+1, format='csr')
    PM = projectionMatrix(params)
    for i in range(params['N_sites']):
        nop, n2op = numOp(params, site_no = i)
        if i!=params['N_sites']-1:
            H = H + (hopOp(params, site_no = i)+hopOp(params, site_no = i).conj().T)*params['J'] 
        H = H + 0.5*params['U']*(n2op - nop)
    
    H = PM@H@PM.transpose()
    _ , eigenvec = sp.linalg.eigsh(H, k=1, which='SA')
    return H, eigenvec

In [78]:
n_max = params['n_max']
theta = np.pi
creation_op = sp.diags(np.sqrt(np.arange(1, n_max+1, 1)), -1, format='csr')
number_op = sp.diags(np.arange(0, n_max+1, 1), 0, format='csr')
identity_op = sp.eye(n_max+1, format='csr')

# Exponential factor e^{i θ n_j}
exp_factor = sp.linalg.expm(1j * theta * number_op)
print(exp_factor.toarray())

[[ 1.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j -1.+1.2246468e-16j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  1.-2.4492936e-16j]]


In [79]:
def parity_op(params, **kwargs):
    '''
    Returns Parity Operator
    '''
    A = sp.diags([(-1.)**n for n in range(params['n_max']+1)])
    return A

In [80]:
parity_op(params).toarray()

array([[ 1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.]])

In [81]:
def prdt_parity_op(params, **kwargs):
    '''
    Returns Parity Operator for a region in BJJ
    Args: params
    '''
    A = parity_op(params)
    start_site, end_site = params['N_region1'], params['N_region2'] 
    I = sp.eye(params['n_max']+1, format='csr')
    lst = [I for _ in range(params['N_sites'])]
    for i in range(start_site, end_site, 1):
        lst[i] = A
    matrixx = reduce(sp.kron, lst)
    return matrixx

In [82]:
def generate_psiPrime(params, **kwargs):
    '''
    Returns wavefunction when Parity Operator for a region is applied on WFn.
    kwargs: start_site, end_site, eigenvec [Required].
    eigenvec: Ground state of BJJ.
    '''
    PM = projectionMatrix(params)
    matrixx = PM@prdt_parity_op(params)@PM.transpose()
    wfn = matrixx@kwargs['eigenvec']
    return wfn

In [84]:
H, eigenvec = two_siteBJJHamil(params)

In [85]:
psi_prime = generate_psiPrime(params, eigenvec = eigenvec)

In [86]:
np.array_equal(psi_prime, eigenvec)

False

In [87]:
def time_evolutionPsiPrime(params, **kwargs):
    '''
    Returns a list of time evolved wavefunctions obtained after the application of Parity Operator of a region 
    for time 0 to T with time step 'tau'.
    This is done wrt the AJJ Hamiltonian.
    kwargs: start_site, end_site, eigenvec, H [Required].
    H: Hamiltonian of AJJ.
    eigenvec: Ground state of AJJ.
    '''
    H, eigenvec = kwargs['H'], kwargs['eigenvec']
    wfn = generate_psiPrime(params, eigenvec = eigenvec)
    time_list = np.arange(0, params['T'], params['tau'])
    unitary_timeOp = [expm(-1j*t*(H.toarray())) for t in time_list]
    psi_t = [normalizeWF(uOp@wfn) for uOp in unitary_timeOp]
    return psi_t